## Training Notebook

This notebook illustrates training of a simple model to classify digits using the MNIST dataset. This code is used to train the model included with the templates. This is meant to be a starter model to show you how to set up Serverless applications to do inferences. For deeper understanding of how to train a good model for MNIST, we recommend literature from the [MNIST website](http://yann.lecun.com/exdb/mnist/). The dataset is made available under a [Creative Commons Attribution-Share Alike 3.0](https://creativecommons.org/licenses/by-sa/3.0/) license.

In [4]:
# We'll use scikit-learn to load the dataset

! pip install -q scikit-learn==0.23.2

In [12]:
# Load the mnist dataset

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, y = fetch_openml('mnist_784', return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000)

In [13]:
# Next, let's add code for deskewing images (we will use this to improve accuracy)
# This code comes from https://fsix.github.io/mnist/Deskewing.html

from scipy.ndimage import interpolation

def moments(image):
    c0, c1 = np.mgrid[:image.shape[0], :image.shape[1]]
    img_sum = np.sum(image)
    
    m0 = np.sum(c0 * image) / img_sum
    m1 = np.sum(c1 * image) / img_sum
    m00 = np.sum((c0-m0)**2 * image) / img_sum
    m11 = np.sum((c1-m1)**2 * image) / img_sum
    m01 = np.sum((c0-m0) * (c1-m1) * image) / img_sum
    
    mu_vector = np.array([m0,m1])
    covariance_matrix = np.array([[m00, m01],[m01, m11]])
    
    return mu_vector, covariance_matrix


def deskew(image):
    c, v = moments(image)
    alpha = v[0,1] / v[0,0]
    affine = np.array([[1,0], [alpha,1]])
    ocenter = np.array(image.shape) / 2.0
    offset = c - np.dot(affine, ocenter)

    return interpolation.affine_transform(image, affine, offset=offset)


def deskew_images(images):
    output_images = []
    
    for image in images:
        output_images.append(deskew(image.reshape(28, 28)).flatten())
    
    return np.array(output_images)


## XGBoost Model Training

For this example, we will train a simple XGBoost classifier using scikit-learn to classify the MNIST digits. We will then freeze the model using `save_model()`. This is same as the starter model file included with the SAM templates.

In [14]:
! pip -q install xgboost==1.1.1

In [17]:
import sklearn
import xgboost as xgb
import numpy as np

from sklearn.metrics import accuracy_score

print('Using versions: sklearn', sklearn.__version__, 'xgboost', xgb.__version__)

# Remove the "tree_method" and "gpu_id" params if you don't have CUDA-compatible GPU set up
xgb_model = xgb.XGBClassifier(n_jobs=-1, tree_method='gpu_hist', gpu_id=0)
xgb_model.fit(X_train, y_train, eval_metric='error')

accuracy = accuracy_score(y_test, xgb_model.predict(np.array(X_test)))

print('Accuracy without deskewing:', accuracy)

Using versions: sklearn 0.23.2 xgboost 1.1.1
Accuracy without deskewing: 0.9782


In [18]:
# Next, we will try with deskewing to see if accuracy improves

# Remove the "tree_method" and "gpu_id" params if you don't have CUDA-compatible GPU set up
xgb_model = xgb.XGBClassifier(n_jobs=-1, tree_method='gpu_hist', gpu_id=0)
xgb_model.fit(deskew_images(X_train), y_train, eval_metric='error')

accuracy = accuracy_score(y_test, xgb_model.predict(deskew_images(X_test)))

print('Accuracy with deskewing:', accuracy)

Accuracy with deskewing: 0.9804


In [19]:
# Save model to the disk
xgb_model.save_model('xgb_digit_classifier.model')